![Demo architecture](img/QPSK_system_block_diagrams_Tx_only.svg)

## Initialisation

Let's include pynq libraries and our own drivers

In [ ]:
import xrfdc
import ipywidgets as ipw

from rfsoc_qpsk import qpsk_overlay, sdr_plots, dma_timer, dict_widget

Now we're ready to load our bitstream to the PL

In [ ]:
ol = qpsk_overlay.QpskOverlay()

### Configuring the RF Data Converters

We are going to use one of the DAC blocks to transmit our signal.
We'll need to generate a clock using one of the on-board synthesizers, tell the data converters about it, and set an NCO frequency --- controlling our signal's carrier frequency. 

![Diagram of on-board synth, DAC block with PLL, and NCO](img/RF_DAC.svg)

Make our RF data converter object

Set DAC's clock source and NCO/mixer frequency

In [ ]:
ol.dac_block.MixerSettings = {
    'CoarseMixFreq':  xrfdc.COARSE_MIX_BYPASS,
    'EventSource':    xrfdc.EVNT_SRC_IMMEDIATE,
    'FineMixerScale': xrfdc.MIXER_SCALE_1P0,
    'Freq':           84.0,
    'MixerMode':      xrfdc.MIXER_MODE_C2R,
    'MixerType':      xrfdc.MIXER_TYPE_FINE,
    'PhaseOffset':    0.0
}

See `xrfdc.py` for implementation, showing how easily we can wrap existing C drivers as object oriented python drivers!

### Initialise our QPSK TX Design

Finally, we make an instance of a driver for the QPSK transmit logic. This controls the signal processing performed on the signal, as well as tapping off the data path, allowing us to interactively plot the intermediate signals.

See `qpsk_tx.py` for the implementation of the driver, exposing registers as object properties using some data-driven generation

## Visualising internal signals

Now that the transmitter is configured and constantly running in the background, we can capture intermediate signals and visualise them interactively.

Our raw, binary data is carried by two parts ("I" and "Q") in a complex signal.  These each have a value of 1 or -1, conveying 1 bit of information.

Let's make a time domain plot of this signal --- straight after our `symbol generation` block.

### Raw QPSK symbols

Here we generate a time domain plot that updates in real-time. We make use of Pynq's DMA drivers, and a python Timer for scheduled transfers.

In [ ]:
cplot = sdr_plots.IQTimePlot(ol.qpsk_tx.get_many_symbols(N=10), 500,
                            resampling_fun=sdr_plots.resample_pick)

dg = dma_timer.DmaTimer(cplot.add_data, ol.qpsk_tx.get_symbols, 0.05)
ipw.VBox([cplot.get_widget(), ipw.HBox(dg.get_widget())])

We can stream in more live data using the play/stop buttons.

Alright, cool! But changing this signal instantaneously from -1 to 1 means there's a near infinite bandwidth (i.e. has components in all frequencies!). To help suppress this for successful transmission, we perform some `pulse shaping`.

### Pulse shaping

Next we make the same plot, but with data from after our pulse shaping unit. First we'll see this in the time domain, then we will look at the same signal in the frequency domain.

In [ ]:
iq_plot = sdr_plots.IQTimePlot(ol.qpsk_tx.get_many_shaped_time(N=10), 4000,w=800)
iq_dg = dma_timer.DmaTimer(iq_plot.add_data, ol.qpsk_tx.get_shaped_time, 0.05)

fa_plot = sdr_plots.HWFreqPlot(ol.qpsk_tx.get_shaped_fft(), 4000, avg_n=4,w=800)
fa_dg = dma_timer.DmaTimer(fa_plot.add_frame, ol.qpsk_tx.get_shaped_fft, 0.3)

tab1 = ipw.Tab([ipw.VBox([iq_plot.get_widget(), ipw.HBox(iq_dg.get_widget())]),
                ipw.VBox([fa_plot.get_widget(), ipw.HBox(fa_dg.get_widget())])
               ])
tab1.set_title(0, 'Time domain')
tab1.set_title(1, 'Frequency domain')
tab1

## Dynamic Control

We can also use widgets to control our PL settings visually. For example, our mixer's NCO frequency and the gain of the signal.

First, let's write some JSON that describes our settings and their valid ranges.

In [ ]:
import json

schema_json = """
{
  "title": "TX Settings",
  "type": "object",
  "properties": {
    "Freq": {
      "description": "NCO Frequency (MHz)",
      "type": "number",
      "minimum": 0.0,
      "maximum": 100.0,
      "default": 84.0
    },
    "Gain": {
      "description": "Relative Gain",
      "type": "number",
      "minimum": 0.0,
      "maximum": 1.0,
      "default": 1
    }
  }
}
"""


Now, let's define a function that will take a dictionary of these settings and apply them to our system. We must ensure that only one instance of this is run at a given time (using Python's `Lock()`) because our driver isn't thread safe (yet).

In [ ]:
from threading import Thread, Lock

tx_mutex = Lock()

def update_tx_settings(config):
    # Acquire lock for mutual exclusion on RF driver
    if tx_mutex.acquire(blocking=False):
        if 'Gain' in config:
            ol.qpsk_tx.axi_qpsk_tx.output_gain = int(config['Gain']*(2**32-1))
        if 'Freq' in config:
            mixer_cfg = ol.dac_block.MixerSettings
            mixer_cfg['Freq'] = config['Freq']
            ol.dac_block.MixerSettings = mixer_cfg
        tx_mutex.release()

Finally we can make our widget and play with the values.

See the effect of changing the gain in the FFT and time domain signal. The effect of the NCO mixer is a bit more subtle --- keep an eye out for a look of suppressed horror on Kenny's face as his receiver gets out of sync!

In [ ]:
cfg = dict_widget.DictWidget(json.loads(schema_json))
gui = cfg.interact(callback=update_tx_settings)
display(gui)

# QPSK Receiver

Now that the transmitter is configured, we start to play with the receiver side.

![Demo architecture](img/QPSK_system_block_diagrams_Rx_only.svg)

## Config

First, we configure the RF ADC block. The ADC tile's PLL is configured, and make sure to match the block's mixer frequency with the transmitting side! For us that's 84 MHz unless you've been playing with the sliders above...

In [ ]:
ol.adc_block.MixerSettings = {
    'CoarseMixFreq':  xrfdc.COARSE_MIX_BYPASS,
    'EventSource':    xrfdc.EVNT_SRC_TILE,
    'FineMixerScale': xrfdc.MIXER_SCALE_1P0,
    'Freq':           84.0,
    'MixerMode':      xrfdc.MIXER_MODE_R2C,
    'MixerType':      xrfdc.MIXER_TYPE_FINE,
    'PhaseOffset':    0.0
}
ol.adc_block.UpdateEvent(xrfdc.EVENT_MIXER)

## Decimated signal

Now we can start inspecting some of the received signals.
Below, we can grab data from multiple parts of our RX signal path and plot them in the time domain, frequency domain, or as a constellation.

Feel free to change the first line to swap between inspecting the raw signal after decimation, after coarse synchronisation, or after the RRC.

In [ ]:
taps = {
    'Decimated': 
      {'get': ol.qpsk_rx.get_decimated,     'get_many': ol.qpsk_rx.get_many_decimated,    'fs':1024},
    'RRCed': 
      {'get': ol.qpsk_rx.get_rrced,         'get_many': ol.qpsk_rx.get_many_rrced,         'fs':16384},
    'CoarseSynced': 
      {'get': ol.qpsk_rx.get_coarse_synced, 'get_many': ol.qpsk_rx.get_many_coarse_synced, 'fs':4096}
}

In [ ]:
tap = 'Decimated'
#     ^^^^^^^^^^^
#    Pick between Decimated, CoarseSynced, RRCed, or Data

In [ ]:
f=taps[tap]['get']
fm=taps[tap]['get_many']
fs=taps[tap]['fs']

f_plot = sdr_plots.IQFreqPlot(fm(), fs)
f_dg = dma_timer.DmaTimer(f_plot.add_frame, fm, 0.3)

iq_plot = sdr_plots.IQTimePlot(fm(), fs, w=800)
iq_dg = dma_timer.DmaTimer(iq_plot.add_data, f, 0.05)

c_plot = sdr_plots.IQConstellationPlot(fm(), plotrange=(0, len(fm())-1), fade=True)
c_dg = dma_timer.DmaTimer(c_plot.add_data, f, 0.05)

tab1 = ipw.Tab([ipw.VBox([iq_plot.get_widget(), ipw.HBox(iq_dg.get_widget())]),
                ipw.VBox([f_plot.get_widget(), ipw.HBox(f_dg.get_widget())]),
                ipw.VBox([c_plot.get_widget(), ipw.HBox(c_dg.get_widget())]),])
tab1.set_title(0, 'Time domain')
tab1.set_title(1, 'Frequency domain')
tab1.set_title(2, 'Constellation')
tab1

# Output data

Finally we can plot a constellation of the final, synchronised signal, showing the 4 constellations expected of QPSK. The transparency of each sample represents its "age", as there in no explicit time axis for constellation plots. On our test system, ~20 frames per second is achievable.

In [ ]:
d=ol.qpsk_rx.get_many_data()
fs=500

iq_plot = sdr_plots.IQTimePlot(d, fs, w=800)
iq_dg = dma_timer.DmaTimer(iq_plot.add_data, ol.qpsk_rx.get_data, 0.05)

c_plot = sdr_plots.IQConstellationPlot(d, plotrange=(0, len(d)-1), fade=True)
c_dg = dma_timer.DmaTimer(c_plot.add_data, ol.qpsk_rx.get_data, 0.05)

tab1 = ipw.Tab([ipw.VBox([c_plot.get_widget(), ipw.HBox(c_dg.get_widget())]),
                ipw.VBox([iq_plot.get_widget(), ipw.HBox(iq_dg.get_widget())])])
tab1.set_title(0, 'Constellation')
tab1.set_title(1, 'Time domain')
tab1